# 01. 라벨링 툴
### OpenCV로 영상 보면서 강조 구간 라벨링
- SPACE: 강조 구간 시작/끝 토글
- q: 종료 + CSV 저장

In [ ]:
import cv2
import csv
import os

from config import  ensure_dirs
ensure_dirs()

In [ ]:
VIDEO_PATH = "/home/piai/바탕화면/영상raw/날씨1.mp4"
OUTPUT_CSV = "/home/piai/바탕화면/영상csv/날씨1.csv" 
VIDEO_ID = "v001"

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"비디오를 열 수 없습니다: {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"fps: {fps}, total_frames: {total_frames}")

fps: 29.922, total_frames: 2268


In [ ]:
segments = []          
marking = False
start_time = None

cv2.namedWindow("Emphasis Labeling", cv2.WINDOW_NORMAL)

print("사용법:")
print("  SPACE : 강조 구간 시작/끝 토글")
print("  q     : 종료 후 CSV 저장")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cur_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
    cur_sec = cur_ms / 1000.0

    disp = frame.copy()
    time_text = f"time: {cur_sec:6.2f}s"
    color = (0, 0, 255) if marking else (0, 255, 0)
    cv2.putText(disp, time_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2, cv2.LINE_AA)

    if marking:
        msg = "MARKING EMPHASIS (SPACE = END, q = QUIT)"
        cv2.putText(disp, msg, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
    else:
        msg = "SPACE = START EMPHASIS, q = QUIT"
        cv2.putText(disp, msg, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Emphasis Labeling", disp)

    key = cv2.waitKey(30) & 0xFF

    if key == ord(' '):
        if not marking:
            marking = True
            start_time = cur_sec
            print(f"[START] emphasis at {start_time:.2f}s")
        else:
            marking = False
            end_time = cur_sec
            if end_time > start_time:
                segments.append((start_time, end_time, 1))
                print(f"[END]   emphasis at {end_time:.2f}s  (len={end_time-start_time:.2f}s)")
            else:
                print("[WARN] end_time <= start_time, 구간 무시")
    elif key == ord('q'):
        print("사용자 종료(q)")
        break

if marking and start_time is not None:
    end_time = total_frames / fps
    segments.append((start_time, end_time, 1))
    print(f"[AUTO END] 마지막 구간을 {end_time:.2f}s까지로 저장")

cap.release()
cv2.destroyAllWindows()

사용법:
  SPACE : 강조 구간 시작/끝 토글
  q     : 종료 후 CSV 저장
[START] emphasis at 16.48s
[END]   emphasis at 19.68s  (len=3.21s)
[START] emphasis at 28.61s
[END]   emphasis at 34.66s  (len=6.05s)
[START] emphasis at 39.20s
[END]   emphasis at 41.41s  (len=2.21s)
[START] emphasis at 45.42s
[END]   emphasis at 47.59s  (len=2.17s)
[START] emphasis at 48.43s
[END]   emphasis at 51.43s  (len=3.01s)
[START] emphasis at 54.01s
[END]   emphasis at 58.35s  (len=4.34s)
사용자 종료(q)


In [ ]:
print("총 강조 구간 수:", len(segments))
for i, (s, e, _) in enumerate(segments):
    print(f"  {i}: {s:.2f}s ~ {e:.2f}s")

총 강조 구간 수: 6
  0: 16.48s ~ 19.68s
  1: 28.61s ~ 34.66s
  2: 39.20s ~ 41.41s
  3: 45.42s ~ 47.59s
  4: 48.43s ~ 51.43s
  5: 54.01s ~ 58.35s


In [ ]:
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)

with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["video_id", "video_path", "start_sec", "end_sec", "label"])
    for (s, e, lab) in segments:
        writer.writerow([VIDEO_ID, VIDEO_PATH, f"{s:.3f}", f"{e:.3f}", lab])

print(f"CSV 저장 완료: {OUTPUT_CSV}")

CSV 저장 완료: /home/piai/바탕화면/영상csv/날씨1.csv
